### Record linkage with Splink

This notebook provides a short worked example using the Splink [quick and dirty persons model](https://moj-analytical-services.github.io/splink/demos/examples/duckdb/quick_and_dirty_persons.html).

In [1]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_library as cl
import pandas as pd

I'm using a slightly modified dataset of [Miami-Dade County Jail Bookings](https://gis-mdc.opendata.arcgis.com/datasets/c2275711ced240c6bc4e998ee1910e85). 

The dataset used in this notebook can also be found within this GitHub repository. There are 266,656 unique forename, surname and dob combinations within the dataset.

In [2]:
df = pd.read_csv('data_md_jb_v2.csv', low_memory=False)
df.head()

,unique_id,book_date,date_eu,defendant,surname,first_name,address,city_state_zip,dob,charge1,charge2,charge3,zip,city,state
0,322567,2021/12/01 05:00:00+00,01/12/2021,GERT ALLYSSON,GERT,ALLYSSON,JAKOBFUGLISTRASSE 18 804,ZURICH YY,1995-06-02,CONT SUBS/POSS,CONT SUBS/POSS,DRUG PARAPHERNA/POSN,NaN,ZURICH,YY
1,402605,2023/09/29 04:00:00+00,29/09/2023,THOMAS COLMY,THOMAS,COLMY,1368 JAMES CT,ZIONVILLE IN 46077,1981-07-05,OUT-OF-CNTY/WARRANT,NaN,NaN,46077,ZIONVILLE,IN
2,300730,2021/05/29 04:00:00+00,29/05/2021,FARMER CORNELL L,FARMER CORNELL,L,2209 ESHCOL AVE,ZION FL 60099,1988-08-20,BATTERY,NaN,NaN,60099,ZION,FL
3,78183,2016/09/24 04:00:00+00,24/09/2016,SCHAPPERT COLE,SCHAPPERT,COLE,1106 CARDINAL DRIVE,ZION IL,1991-05-28,UTTER FORGED INSTRU,RESIST OFF W/O VIOL,DEBIT CARD/UNLAW/USE,NaN,ZION,IL
4,377120,2023/03/09 05:00:00+00,09/03/2023,BENNETT TERRY T,BENNETT TERRY,T,1109 PHEASANT RUN,ZION IL 60099,1992-06-05,SMO/CAN/M/HE/PP/PROH,NaN,NaN,60099,ZION,IL


In [5]:
unique_persons_raw = df.drop_duplicates(subset=['defendant', 'dob']).shape[0]
print("Unique number of individuals in raw dataset by combining name and dob':", unique_persons_raw)

Unique number of individuals in raw dataset by combining name and dob': 266656


### 1.Settings

I've modified the suggested settings in the Splink walkthrough to reflect the columns available in the sample dataset.

A full guide on the blocking rules can be found at this link: https://moj-analytical-services.github.io/splink/topic_guides/blocking/blocking_rules.html

In [6]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        block_on("defendant"),
        block_on(["substr(defendant,1,6)", "dob"]),
    ],
    "comparisons": [
        cl.jaro_at_thresholds("defendant", [0.9, 0.7], term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("dob", [1, 2]),
    ],       

}

### 2. Parameters

See here: https://moj-analytical-services.github.io/splink/demos/tutorials/04_Estimating_model_parameters.html

In [7]:
# these are unchanged from SplinkQD documentation tutorial
linker = DuckDBLinker(df, settings, set_up_basic_logging=False)
deterministic_rules = [
    "l.defendant = r.defendant",
    "l.defendant = r.defendant and l.dob = r.dob",
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

linker.estimate_u_using_random_sampling(max_pairs=2e6)

### 3. Results

In [9]:
results = linker.predict(threshold_match_probability=0.75)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'defendant':
    m values not fully trained
Comparison: 'dob':
    m values not fully trained


In [10]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,defendant_l,defendant_r,gamma_defendant,dob_l,dob_r,gamma_dob,match_key
0,1.68325,0.762555,4922,11063,MORILLO RICHARD,MORILLO RICHARD,3,1977-09-19,1977-09-17,2,0
1,1.68325,0.762555,3854,11063,MORILLO RICHARD,MORILLO RICHARD,3,1977-09-19,1977-09-17,2,0
2,1.68325,0.762555,264718,267793,LEEMAN COURTNEY BROCK,LEEMAN COURTNEY BROCK,3,1979-02-23,1979-06-23,2,0
3,1.68325,0.762555,258668,267793,LEEMAN COURTNEY BROCK,LEEMAN COURTNEY BROCK,3,1979-02-23,1979-06-23,2,0
4,1.68325,0.762555,165746,267793,LEEMAN COURTNEY BROCK,LEEMAN COURTNEY BROCK,3,1979-02-23,1979-06-23,2,0
5,1.68325,0.762555,563,133317,CASTRO MELISSA,CASTRO MELISSA,3,1988-06-08,1980-06-08,2,0
6,1.68325,0.762555,101659,146990,BANEGAS JOSUE GABRIEL,BANEGAS JOSUE GABRIEL,3,1986-01-14,1981-01-14,2,0
7,1.68325,0.762555,101659,162534,BANEGAS JOSUE GABRIEL,BANEGAS JOSUE GABRIEL,3,1986-01-14,1981-01-14,2,0
8,1.68325,0.762555,101659,204779,BANEGAS JOSUE GABRIEL,BANEGAS JOSUE GABRIEL,3,1986-01-14,1981-01-14,2,0
9,1.68325,0.762555,101659,207043,BANEGAS JOSUE GABRIEL,BANEGAS JOSUE GABRIEL,3,1986-01-14,1981-01-14,2,0


### 4. Adding unique id to persons and using results

SpinkQD identified 27,490 duplicate individuals from our original dataset. 

In [14]:
#add in cluster id
clusters = linker.cluster_pairwise_predictions_at_threshold(results, threshold_match_probability=0.5)
clusters.as_pandas_dataframe(limit=5)

,cluster_id,unique_id,book_date,date_eu,defendant,surname,first_name,address,city_state_zip,dob,charge1,charge2,charge3,zip,city,state,__splink_salt,tf_defendant
0,19986,297447,2021/04/29 04:00:00+00,29/04/2021,SCOTT LARRY,SCOTT,LARRY,ADDRESS UNKNOWN,MIAMI FL,1959-03-20,ALC/BEV CONSUME/SERV,None,None,None,MIAMI,FL,0.068558,0.000024
1,105241,297720,2021/05/04 04:00:00+00,04/05/2021,BROWN DERIK LAMOYNE,BROWN DERIK,LAMOYNE,15020 TYLER ST,MIAMI FL 33176,1986-11-27,BATTERY/AGGRAVATED,CHILD ABUSE/NO HARM,CHILD ABUSE/NO HARM,33176,MIAMI,FL,0.849602,0.000024
2,21799,298104,2021/05/08 04:00:00+00,08/05/2021,MASSEY PIERRE ALEXANDER,MASSEY PIERRE,ALEXANDER,ADDRESS UNKNOWN,MIAMI FL,1973-03-18,PETIT THEFT 2> CONV,None,None,None,MIAMI,FL,0.801492,0.000041
3,298242,298242,2021/05/07 04:00:00+00,07/05/2021,STANFORD DELVON,STANFORD,DELVON,3094 E LIVINGSTON AVE,COLUMBUS OH 43227,1998-09-23,CANNABIS/PURCHASE,None,None,43227,COLUMBUS,OH,0.542894,0.000002
4,130110,298745,2021/05/11 04:00:00+00,11/05/2021,LEWIS CAMILLE DANYELL,LEWIS CAMILLE,DANYELL,1525 NW 60TH ST 6,MIAMI FL 33142,1986-12-21,BATTERY,None,None,33142,MIAMI,FL,0.500077,0.000002
5,229709,298749,2021/05/11 04:00:00+00,11/05/2021,LINARES ALEXANDER,LINARES,ALEXANDER,2509 W 72ND ST,HIALEAH FL 33016,1971-03-30,DWLS/HABITUAL,None,None,33016,HIALEAH,FL,0.456094,0.000019
6,47570,298750,2021/05/10 04:00:00+00,10/05/2021,SMITH GREGORY KEITH,SMITH GREGORY,KEITH,4800 NW 170 ST.ERR,MIAMI GAREENS FL 33055,1966-06-23,ALIAS CAPIAS,None,None,33055,MIAMI GAREENS,FL,0.580879,0.000005
7,274729,298767,2021/05/11 04:00:00+00,11/05/2021,MARSHALL LEXIE NATINCE,MARSHALL LEXIE,NATINCE,1811 SW 48 AVE,WEST PARK FL 33023,1997-04-25,DWLS/KNOWINGLY,None,None,33023,WEST PARK,FL,0.829338,0.000005
8,46127,298806,2021/05/13 04:00:00+00,13/05/2021,GONZALEZDELAPENA LUIS ALBERTO,GONZALEZDELAPENA LUIS,ALBERTO,385 WEST 46 ST,HIALEAH FL,1997-07-01,CANNABIS/POSN/0-20,DRUG PARAPHERNA/POSN,VEH REG/NOT HAVE,None,HIALEAH,FL,0.781381,0.000002
9,75682,298814,2021/05/15 04:00:00+00,15/05/2021,FERNANDEZ FRANCISCO,FERNANDEZ,FRANCISCO,155 E 11TH ST,HIALEAH FL 33010,1985-12-10,RET TFT/300>/BF 10/1,RET TFT/300>/BF 10/1,None,33010,HIALEAH,FL,0.031745,0.000036


In [17]:
clusters.to_csv('cluster_results.csv', overwrite=True, low_memory=False)
results_df = pd.read_csv('cluster_results.csv')

unique_persons_splinkQD = results_df.drop_duplicates('cluster_id').shape[0]
print("Unique number of individuals after Splink Quick and Dirty':", unique_persons_splinkQD)

C:\Users\agari\AppData\Local\Temp\ipykernel_5844\1666739761.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  results_df = pd.read_csv('cluster_results.csv')


Unique number of individuals after Splink Quick and Dirty': 239166


In [18]:
difference = unique_persons_raw - unique_persons_splinkQD
print("Number of duplicate persons found in raw data':", difference)

Number of duplicate persons found in raw data': 27490
